In [1]:
import os  # type:ignore # isort:skip # fmt:skip # noqa # nopep8
import sys  # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from pathlib import Path  # type:ignore # isort:skip # fmt:skip # noqa # nopep8

mod = sys.modules[__name__]

code_dir = None
code_dir_name = 'Code'
unwanted_subdir_name = 'Analysis'

for _ in range(5):

    parent_path = str(Path.cwd().parents[_]).split('/')[-1]

    if (code_dir_name in parent_path) and (unwanted_subdir_name not in parent_path):

        code_dir = str(Path.cwd().parents[_])

        if code_dir is not None:
            break

sys.path.append(code_dir)
# %load_ext autoreload
# %autoreload 2


In [2]:
from setup_module.imports import *  # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from setup_module import researchpy_fork as rp # type:ignore # isort:skip # fmt:skip # noqa # nopep8
from setup_module import specification_curve_fork as specy # type:ignore # isort:skip # fmt:skip # noqa # nopep8


Using MPS


0it [00:00, ?it/s]

<Figure size 640x480 with 0 Axes>

# Functions

In [3]:
# Function to order categories
def categorize_df_results_gender_age(df, gender_order=None, age_order=None, ivs=None):
    if gender_order is None:
        gender_order = ['Female', 'Mixed Gender', 'Male']
    if age_order is None:
        age_order = ['Older', 'Mixed Age', 'Younger']
    if ivs is None:
        ivs = ['Gender', 'Age']
    # Arrange Categories
    for iv in ivs:
        if iv == 'Gender':
            order = gender_order
        elif iv == 'Age':
            order = age_order
        try:
            df[iv] = df[iv].astype('category').cat.reorder_categories(order, ordered=True)

            df[iv] = pd.Categorical(
                df[iv], categories=order, ordered=True
            )
            df[f'{iv}_Num'] = pd.to_numeric(df[iv].cat.codes).astype('int64')
        except ValueError as e:
            print(e)

    return df


# READ DATA

In [4]:
with open(f'{data_dir}df_manual_len.txt', 'r') as f:
    df_manual_len = int(f.read())

df_manual = pd.read_pickle(f'{df_save_dir}df_manual_for_training.pkl')
assert len(df_manual) == df_manual_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_manual_len} BUT IS OF LENGTH {len(df_manual)}'
print(f'Dataframe loaded with shape: {df_manual.shape}')


Dataframe loaded with shape: (5947, 68)


In [5]:
with open(f'{data_dir}df_jobs_len.txt', 'r') as f:
    df_jobs_len = int(f.read())

# df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_for_analysis.pkl')
df_jobs = pd.read_pickle(f'{df_save_dir}df_jobs_for_analysis.pkl')
assert len(df_jobs) == df_jobs_len, f'DATAFRAME MISSING DATA! DF SHOULD BE OF LENGTH {df_jobs_len} BUT IS OF LENGTH {len(df_jobs)}'
print(f'Dataframe loaded with shape: {df_jobs.shape}')


Dataframe loaded with shape: (307300, 83)


In [6]:
df_manual = categorize_df_results_gender_age(df_manual)

In [7]:
df_jobs = categorize_df_results_gender_age(df_jobs)

## Set dataframes

In [8]:
dataframes = {
    'df_manual': df_manual,
    'df_jobs': df_jobs,
}

## Analysis plan:

1. [Descriptives, visualizations, and tables](./1.%20descriptives_visualization_and_tables.ipynb)
2. [Frequencies and Normality tests](./2.%20frequencies_and_normality_test.ipynb)
   1. Frequencies, histograms, and QQ plots
      * Normal test
      * Kurtosis test
      * Shapiro
      * Anderson
      * Bartlett
   2. Correlation between independent variables (ivs) and control variables and Multicolinarity test
      * Pearson's R
      * VIF
     - **ivs_dummy** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
     - **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
     - **% Sector per Workforce** (continous ratio) = Sector percentage per worksforce (0-100)
     - **num_words** (continous ratio) = Number of words in job description
     - **English Requirement in Sentence** (binary nominal) = English requirement in job description (0 vs. 1)
     - **Dutch Requirement in Sentence** (binary nominal) = Dutch requirement in job description (0 vs. 1)

3. [Secondary Analysis](./3.%20chisqt_anova_and_regression.ipynb)

   1. Chi-square
      * df_manual:
        - **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
        - **ivs** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
      * df_jobs:
        - **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
        - **ivs** (binary nominal) = Social category designation (Female, Male, Mixed Gender)

   2. One-way ANOVA, interactions, and post-hoc test
      * df_manual:
        - **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
        - **ivs** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
          - If Levene's test is *not significant*, use classic ANOVA and Tukey's post hoc test
          - If Levene's test is *significant*, use Welch's and Kruskal-Wallis ANOVA and Games Howell's post hoc test
      * df_jobs:
         - **dvs_prob** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
         - **ivs** (binary nominal) = Social category designation (Female, Male, Mixed Gender)
           - If Levene's test is *not significant*, use classic ANOVA and Tukey's post hoc test
           - If Levene's test is *significant*, use Welch's and Kruskal-Wallis ANOVA and Games Howell's post hoc test

   3. Logistic Regression  with all interaction (smf):
      * df_manual:
        - **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
        - **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
      * df_jobs:
        - **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
        - **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
   4. OLS Regression with all interaction:
      * df_jobs:
        - **dvs_prob** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
        - **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
   5. Multilevel OLS Regression with all interaction:
      * df_jobs:
        - **dvs_prob** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
        - **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)

4. [Main Analysis](./4.%20specification_curve_analysis.ipynb)

   1. Logistic Specification Curve Analysis:
      * df_manual:
        - **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
        - **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
       * df_jobs:
        - **dvs** (binary nominal) = 'Warmth' and 'Competence' (0 vs. 1)
        - **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)
   2. OLS Specification Curve Analysis:
      * df_jobs:
        - **dvs_prob** (continous ratio) = 'Warmth' and 'Competence' probabilities (0-1)
        - **ivs_perc** (continous ratio) = Social category percentage per sector (0-100)


# Frequencies


In [9]:
for df_name, df in dataframes.items():
    print(f'{"="*5} RESULTS FOR {df_name} {"="*5}')

    data_names = dvs[:]+ivs_dummy_and_perc[:]

    if df_name == 'df_manual':
        dvs_ = dvs[:]
    elif df_name == 'df_jobs':
        data_names.extend(dvs_prob[:])
        dvs_ = dvs_all[:]

    print('~'*20)
    # print(rp.codebook(df[data_names]))

    # Gender and Age
    print('-'*20)
    print(f'Categorical Summary {ivs}')
    freq_iv=rp.summary_cat(df[ivs]).round(3)
    print(freq_iv)
    freq_iv.to_csv(f'{table_save_path}frequencies {df_name} - Gender and Age.csv')
    print('-'*20)
    print('\n')

    # Gender and Age Percentages
    print('-'*20)
    print(f'Continuous Summary {ivs_perc}')
    freq_iv_perc=rp.summarize(df[ivs_perc], ci_level = 0.95, decimals = 3)
    print(freq_iv_perc)
    freq_iv_perc.to_csv(f'{table_save_path}frequencies {df_name} - Gender and Age Percentages.csv')
    print('-'*20)
    print('\n')

    # Gender and Age Percentages
    print('-'*20)
    print(f'Continuous Summary {ivs_count}')
    freq_iv_count=rp.summarize(df[ivs_count], ci_level = 0.95, decimals = 3)
    print(freq_iv_count)
    freq_iv_count.to_csv(f'{table_save_path}frequencies {df_name} - Gender and Age Counts.csv')
    print('-'*20)
    print('\n')

    # Warmth and Competence
    print('-'*20)
    print(f'Categorical Summary {dvs}')
    freq_dv=rp.summary_cat(df[dvs]).round(3)
    print(freq_dv)
    freq_dv.to_csv(f'{table_save_path}frequencies {df_name} - Warmth and Competence.csv')
    print('-'*20)
    print('\n')

    if df_name == 'df_jobs':
        # Warmth and Competence Probabilities
        print('-'*20)
        print(f'Continuous Summary {dvs_prob}')
        freq_dv_prob=rp.summarize(df[dvs_prob], ci_level = 0.95, decimals = 3)
        print(freq_dv_prob)
        freq_dv_prob.to_csv(f'{table_save_path}frequencies {df_name} - Warmth and Competence Probabilities.csv')
        print('-'*20)
        print('\n')

    print('-'*20)
    print('Grouped Frequencies/ Summary ANOVAs')
    summary_aova = rp.summary_cont(df.groupby(ivs)[dvs], conf=0.95, decimals=3)
    print(summary_aova)
    summary_aova.to_csv(f'{table_save_path}summary anova {df_name} - {ivs} x {dvs}.csv')
    print('-'*20)
    print('\n')

    if df_name == 'df_jobs':
        print('-'*20)
        print('Grouped Frequencies/ Summary ANOVAs')
        summary_aova_probs = rp.summary_cont(df.groupby(ivs)[dvs_all], conf=0.95, decimals=3)
        print(summary_aova_probs)
        summary_aova_probs.to_csv(f'{table_save_path}summary anova probabilities {df_name} - {ivs} x {dvs_all}.csv')
        print('-'*20)
        print('\n')

    # # Histogram
    # df[ivs_perc].hist()
    # plt.ion()
    # plt.show()
    # plt.clf()
    # plt.cla()
    # plt.close()
    # print('-'*20)
    # print('\n')

    # df[ivs_count].hist()
    # plt.ion()
    # plt.show()
    # plt.clf()
    # plt.cla()
    # plt.close()
    # print('-'*20)
    # print('\n')

    # df[ivs_dummy].hist()
    # plt.ion()
    # plt.show()
    # plt.clf()
    # plt.cla()
    # plt.close()
    # print('-'*20)
    # print('\n')

    # if df_name == 'df_jobs':
    #     # Histogram
    #     df[dvs_prob].hist()
    #     plt.ion()
    #     plt.show()
    #     plt.clf()
    #     plt.cla()
    #     plt.close()
    #     print('-'*20)
    #     print('\n')

    # # QQ plot
    # qq_plot = pg.qqplot(df[ivs_perc], dist='norm')
    # plt.ion()
    # plt.show()
    # plt.clf()
    # plt.cla()
    # plt.close()
    # print('-'*20)
    # print('\n')

    # qq_plot = pg.qqplot(df[ivs_count], dist='norm')
    # plt.ion()
    # plt.show()
    # plt.clf()
    # plt.cla()
    # plt.close()
    # print('-'*20)
    # print('\n')

    # qq_plot = pg.qqplot(df[ivs_dummy], dist='norm')
    # plt.ion()
    # plt.show()
    # plt.clf()
    # plt.cla()
    # plt.close()
    # print('-'*20)
    # print('\n')

    # if df_name == 'df_jobs':
    #     # QQ plot dvs_prob
    #     qq_plot = pg.qqplot(df[dvs_prob], dist='norm')
    #     plt.ion()
    #     plt.show()
    #     plt.clf()
    #     plt.cla()
    #     plt.close()
    #     print('-'*20)
    #     print('\n')


===== RESULTS FOR df_manual =====
~~~~~~~~~~~~~~~~~~~~
--------------------
Categorical Summary ['Gender', 'Age']
  Variable    Outcome     Count  Percent
0  Gender   Mixed Gender  3836   64.50  
1                   Male  1151   19.35  
2                 Female   960   16.14  
3     Age        Younger  3221   54.16  
4              Mixed Age  2045   34.39  
5                  Older   681   11.45  
--------------------


--------------------
Continuous Summary ['Gender_Female_% per Sector', 'Gender_Male_% per Sector', 'Age_Older_% per Sector', 'Age_Younger_% per Sector']
              Name              N    Mean Median Variance   SD   SE       95% Conf. Interval    
0  Gender_Female_% per Sector  5947 45.15 42.75   252.27  15.88 0.21  Gender_Female_% per Sector
1    Gender_Male_% per Sector  5947 54.83 57.25   253.02  15.91 0.21    Gender_Male_% per Sector
2      Age_Older_% per Sector  5947 38.12 31.96    76.39   8.74 0.11      Age_Older_% per Sector
3    Age_Younger_% per Sector  5947

# Normality Tests


In [10]:
for df_name, df in dataframes.items():

    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')
    if df_name == 'df_manual':
        dvs_ = dvs
    elif df_name == 'df_jobs':
        dvs_ = dvs_all

    # Test of Normality for skew and kurtosis
    # if p < alpha, the null hypothesis is rejected, x is not from a normal distribution
    norm = scipy.stats.normaltest(df[dvs_])
    print('='*80)
    print(f'{dvs_} Test of Normality:')
    print('-'*80)
    for key, val in dict(zip(normality_tests_labels, norm)).items():
        print(key,': ', val) # not significant
    print('\n')

    # Shapir-Wilk Test of Normality
    # if p < alpha, the null hypothesis is rejected, x is not from a normal distribution
    norm_res = scipy.stats.shapiro(df[dvs_])
    print('='*80)
    print(f'{dvs_} Shapir-Wilk Test of Normality:')
    print('-'*80)
    for key, val in dict(zip(normality_tests_labels, norm_res)).items():
        print(key,': ', val) # significant
    print('\n')

    for iv, dv in tqdm_product(ivs_dummy_and_perc, dvs_):
        print('+'*120)
        print(f'Dependent Variable: {dv}\nIndependent Variable: {iv}')
        print('+'*120)

        # Anderson-Darling Test of Normality
        # if p < alpha, the null hypothesis is rejected, x is not from a normal distribution
        norm_and = scipy.stats.anderson(df[dv])
        print('='*80)
        print('Anderson-Darling Test of Normality:')
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        for key, val in dict(zip(normality_tests_labels, norm_and)).items():
            print(key,': ', val) # not significant
        print('\n')
        if norm_and.fit_result.success:
            print('Anderson-Darling Test of Normality: The test was successful.')
        else:
            print('Anderson-Darling Test of Normality: The test was not successful.')
        print('~' * 20)
        print('\n')

        # NORMALITY TESTS
        print('NORMALITY TEST')
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        norm = pg.normality(data=df, dv=dv, group=iv).round(3)
        normal = bool(norm['normal'].to_string(index=False))
        print(f"{iv} x {dv} Normality test:\n{norm}")
        norm.to_csv(f"{table_save_path}normality {df_name} - {iv} x {dv}.csv")
        print('~' * 20)
        print('\n')

        # # ANOVA SPHERICITY TEST
        # print('SPHERICITY TEST')
        # print('\n')
        # print('~' * 20)
        # print(f'{iv} x {dv}')
        # spher_all = pg.sphericity(data=df, dv=dv, within=iv, method='mauchly')
        # spher, test_stat, chisq, dof, pval = spher_all
        # print('-' * 20)
        # print(f"{iv} x {dv} Sphericity test:\n{spher} at p-value: {round(pval, 3)}, chi-square: {round(chisq, 3)}, degrees of freedom: {round(dof)}, Test statistic: {round(test_stat)}") # if p-value < 0.05, then the data are not spherically distributed = Multivariate analysis
        # # spher.to_csv(f"{table_save_path}sphericity {df_name} - {iv} x {dv}.csv")
        # print('~' * 20)
        # print('\n')

        # BARTLETTS TESTS
        print("BARTLETT'S TEST")
        print('\n')
        print('~' * 20)
        print(f'{iv} x {dv}')
        bartlett = pg.homoscedasticity(data=df, dv=dv, group=iv, method='bartlett').round(3) #dv
        equal_var_bartlett = bool(bartlett['equal_var'].to_string(index=False))
        print(f"{iv} x {dv} Bartlett's test:\n{bartlett}")
        bartlett.to_csv(f"{table_save_path}bartlett's {df_name} - {iv} x {dv}.csv")
        print('~' * 20)
        print('\n')



++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_manual ==================================================
['Warmth', 'Competence'] Test of Normality:
--------------------------------------------------------------------------------
Statistic :  [ 1748.11969528 20849.1837926 ]
p-value :  [0. 0.]


['Warmth', 'Competence'] Shapir-Wilk Test of Normality:
--------------------------------------------------------------------------------
Statistic :  0.6109155416488647
p-value :  0.0




/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


  0%|          | 0/20 [00:00<?, ?it/s]

++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Dependent Variable: Warmth
Independent Variable: Gender_Female
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Anderson-Darling Test of Normality:


~~~~~~~~~~~~~~~~~~~~
Gender_Female x Warmth
Statistic :  1367.8821888921857
p-value :  [0.576 0.656 0.786 0.917 1.091]


Anderson-Darling Test of Normality: The test was successful.
~~~~~~~~~~~~~~~~~~~~


NORMALITY TEST


~~~~~~~~~~~~~~~~~~~~
Gender_Female x Warmth
Gender_Female x Warmth Normality test:
                W   pval  normal
Gender_Female                   
1             0.56  0.00   False
0             0.55  0.00   False
~~~~~~~~~~~~~~~~~~~~


BARTLETT'S TEST


~~~~~~~~~~~~~~~~~~~~
Gender_Female x Warmth
Gender_Female x Warmth Bartlett's test:
           T   pval  equal_var
bartlett 0.29  0.59    True   
~~~~~~~~~~~~~~~~~~~~


++++++++++

/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


...

/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.

Age_Younger_% per Sector x Competence Normality test:
                           W   pval  normal
Age_Younger_% per Sector                   
65.92                    0.63  0.00   False
67.83                    0.64  0.00   False
80.81                    0.64  0.00   False
51.18                    0.64  0.00   False
52.98                    0.63  0.00   False
46.89                    0.63  0.00   False
46.21                    0.63  0.00   False
53.73                    0.64  0.00   False
48.53                    0.63  0.00   False
53.14                    0.64  0.00   False
67.24                    0.64  0.00   False
60.55                    0.64  0.00   False
50.61                    0.64  0.00   False
44.44                    0.64  0.00   False
44.83                    0.64  0.00   False
64.26                    0.63  0.00   False
56.82                    0.64  0.00   False
46.72                    0.63  0.00   False
47.18                    0.62  0.00   False
~~~~~~~~~~~~~~~~~~~~



/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.

Age_Younger_% per Sector x Warmth_Probability Normality test:
                           W   pval  normal
Age_Younger_% per Sector                   
65.92                    0.74  0.00   False
67.83                    0.74  0.00   False
80.81                    0.76  0.00   False
51.18                    0.70  0.00   False
52.98                    0.69  0.00   False
46.89                    0.71  0.00   False
46.21                    0.72  0.00   False
53.73                    0.73  0.00   False
48.53                    0.74  0.00   False
53.14                    0.73  0.00   False
67.24                    0.72  0.00   False
60.55                    0.77  0.00   False
50.61                    0.70  0.00   False
44.44                    0.74  0.00   False
44.83                    0.68  0.00   False
64.26                    0.68  0.00   False
56.82                    0.73  0.00   False
46.72                    0.71  0.00   False
47.18                    0.68  0.00   False
~~~~~~~~~~~~~~

/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/scipy/stats/_morestats.py:1816: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")
/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.

## Correlation between IVs and Control Variables (Multicollinearity)

### Categorical Language Requirement

In [11]:
for df_name, df in dataframes.items():
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')

    # Chi-square
    chisqt = pd.crosstab(df[controls[-2]], df[controls[-1]])
    pearson_r, p_value, dof, expected = scipy.stats.chi2_contingency(chisqt)

    print('+'*120)
    print(f'Dependent Variable: {controls[-2]}\nIndependent Variable: {controls[-1]}')
    print('+'*120)
    print('\n')
    print('~' * 20)
    print(f"The Pearsons's R value: {pearson_r:.3f}\nDegree of freedom: {dof}")
    print('-'*20)
    print(f'Observed Count:\n{chisqt}\n')
    print('-'*20)
    print(f'Expected Count:\n{expected}\n')
    print('-'*20)
    print('-' * 20)
    print(f"Pearsons's R p-value: {p_value:.3f}. Rejected: {p_value < 0.05}")
    print('~' * 20)


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_manual ==================================================
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Dependent Variable: Dutch Requirement in Job Ad
Independent Variable: Platform
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++


~~~~~~~~~~~~~~~~~~~~
The Pearsons's R value: 61.542
Degree of freedom: 1
--------------------
Observed Count:
Platform                     Glassdoor  Indeed
Dutch Requirement in Job Ad                   
No                              257      5497 
Yes                              33       160 

--------------------
Expected Count:
[[ 280.58853203 5473.41146797]
 [   9.41146797  183.58853203]]

--------------------
-----------

### VIF for Dummy IVs and Control Variables

In [12]:
# compute the vif for all given features
def compute_vif(df, considered_features):

    X = df[considered_features]
    # the calculation of variance inflation requires a constant
    X.insert(0, 'intercept', 1)

    # create dataframe to store vif values
    vif = pd.DataFrame()
    vif['Variable'] = X.columns
    vif['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    vif = vif.loc[vif['Variable']!='intercept']

    return vif

### VIF for Percentage IVs

In [13]:
for df_name, df in dataframes.items():
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')

    for iv_gender_perc, iv_age_perc in tqdm_product(ivs_gender_perc, ivs_age_perc):
        considered_features = [iv_gender_perc, iv_age_perc] + controls[:7]
        considered_features.remove('Platform_Glassdoor')
        vif = compute_vif(df, considered_features)
        print(vif.sort_values('VIF', ascending=False))


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_manual ==================================================


  0%|          | 0/4 [00:00<?, ?it/s]

/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


               Variable               VIF
1         Gender_Female_% per Sector 1.21
8                    Platform_Indeed 1.17
5  English Requirement in Job Ad_Yes 1.06
3             % Sector per Workforce 1.06
2             Age_Older_% per Sector 1.03
6    Dutch Requirement in Job Ad_Yes 1.02
4          Job Description num_words 1.00
7                  Platform_LinkedIn  NaN


/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


...

               Variable               VIF
1           Gender_Male_% per Sector 1.21
8                    Platform_Indeed 1.17
5  English Requirement in Job Ad_Yes 1.06
3             % Sector per Workforce 1.06
2             Age_Older_% per Sector 1.03
6    Dutch Requirement in Job Ad_Yes 1.02
4          Job Description num_words 1.00
7                  Platform_LinkedIn  NaN


/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


               Variable               VIF
1           Gender_Male_% per Sector 1.21
8                    Platform_Indeed 1.17
5  English Requirement in Job Ad_Yes 1.06
3             % Sector per Workforce 1.06
2           Age_Younger_% per Sector 1.03
6    Dutch Requirement in Job Ad_Yes 1.02
4          Job Description num_words 1.00
7                  Platform_LinkedIn  NaN
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_jobs ==================================================


  0%|          | 0/4 [00:00<?, ?it/s]

               Variable               VIF
7                  Platform_LinkedIn 3.59
8                    Platform_Indeed 3.48
4          Job Description num_words 1.10
3             % Sector per Workforce 1.03
2             Age_Older_% per Sector 1.02
1         Gender_Female_% per Sector 1.02
5  English Requirement in Job Ad_Yes 1.01
6    Dutch Requirement in Job Ad_Yes 1.01
               Variable               VIF
7                  Platform_LinkedIn 3.59
8                    Platform_Indeed 3.48
4          Job Description num_words 1.10
3             % Sector per Workforce 1.03
2           Age_Younger_% per Sector 1.02
1         Gender_Female_% per Sector 1.02
5  English Requirement in Job Ad_Yes 1.01
6    Dutch Requirement in Job Ad_Yes 1.01
               Variable               VIF
7                  Platform_LinkedIn 3.59
8                    Platform_Indeed 3.48
4          Job Description num_words 1.10
3             % Sector per Workforce 1.03
2             Age_Older_% per Sect

### VIF for Categorical Dummy IVs

In [14]:
for df_name, df in dataframes.items():
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')

    considered_features = ivs_dummy[:]
    considered_features.remove('Age_Mixed')
    considered_features.remove('Gender_Mixed')
    vif = compute_vif(df, considered_features)
    print(vif.sort_values('VIF', ascending=False))


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_manual ==================================================
     Variable     VIF
4    Age_Younger 3.30
1  Gender_Female 2.39
2    Gender_Male 2.32
3      Age_Older 1.44
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_jobs ==================================================
     Variable     VIF
2    Gender_Male 1.78
1  Gender_Female 1.63
4    Age_Younger 1.45
3      Age_Older 1.31


In [15]:
for df_name, df in dataframes.items():
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')

    for iv_gender_dummy, iv_age_dummy in tqdm_product(ivs_gender_dummy, ivs_age_dummy):
        print('-'*20)
        print(f'VIF for {iv_gender_dummy} x {iv_age_dummy}')
        considered_features = [iv_gender_dummy, iv_age_dummy] + controls[:7]
        considered_features.remove('Platform_Glassdoor')
        vif = compute_vif(df, considered_features)
        print(vif.sort_values('VIF', ascending=False))


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_manual ==================================================


  0%|          | 0/9 [00:00<?, ?it/s]

--------------------
VIF for Gender_Female x Age_Older


/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


               Variable               VIF
1                      Gender_Female 1.25
8                    Platform_Indeed 1.16
5  English Requirement in Job Ad_Yes 1.08
3             % Sector per Workforce 1.08
2                          Age_Older 1.06
6    Dutch Requirement in Job Ad_Yes 1.02
4          Job Description num_words 1.00
7                  Platform_LinkedIn  NaN
--------------------
VIF for Gender_Female x Age_Mixed


...

/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


               Variable               VIF
2                          Age_Mixed 1.30
1                        Gender_Male 1.19
3             % Sector per Workforce 1.16
8                    Platform_Indeed 1.10
5  English Requirement in Job Ad_Yes 1.05
6    Dutch Requirement in Job Ad_Yes 1.03
4          Job Description num_words 1.01
7                  Platform_LinkedIn  NaN
--------------------
VIF for Gender_Male x Age_Younger
               Variable               VIF
2                        Age_Younger 1.61
1                        Gender_Male 1.61
3             % Sector per Workforce 1.14
8                    Platform_Indeed 1.08
5  English Requirement in Job Ad_Yes 1.03
6    Dutch Requirement in Job Ad_Yes 1.02
4          Job Description num_words 1.01
7                  Platform_LinkedIn  NaN
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_jo

/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


  0%|          | 0/9 [00:00<?, ?it/s]

--------------------
VIF for Gender_Female x Age_Older
               Variable               VIF
7                  Platform_LinkedIn 3.59
8                    Platform_Indeed 3.50
2                          Age_Older 1.15
1                      Gender_Female 1.13
4          Job Description num_words 1.10
3             % Sector per Workforce 1.05
5  English Requirement in Job Ad_Yes 1.01
6    Dutch Requirement in Job Ad_Yes 1.01
--------------------
VIF for Gender_Female x Age_Mixed
               Variable               VIF
7                  Platform_LinkedIn 3.66
8                    Platform_Indeed 3.58
2                          Age_Mixed 1.39
1                      Gender_Female 1.29
3             % Sector per Workforce 1.11
4          Job Description num_words 1.10
5  English Requirement in Job Ad_Yes 1.01
6    Dutch Requirement in Job Ad_Yes 1.01
--------------------
VIF for Gender_Female x Age_Younger
               Variable               VIF
7                  Platform_LinkedI

In [16]:
for df_name, df in dataframes.items():
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')

    considered_features = ivs_dummy[:] + controls[:7]
    considered_features.remove('Age_Mixed')
    considered_features.remove('Gender_Mixed')
    considered_features.remove('Platform_Glassdoor')
    vif = compute_vif(df, considered_features)
    print(vif.sort_values('VIF', ascending=False))
    vif.to_csv(f'{table_save_path}vif {df_name} - {ivs_dummy} x Controls.csv')


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_manual ==================================================


/opt/homebrew/Caskroom/mambaforge/base/envs/study1_3.10/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1752: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss


                Variable               VIF
4                         Age_Younger 3.59
1                       Gender_Female 2.63
2                         Gender_Male 2.51
3                           Age_Older 1.50
5              % Sector per Workforce 1.17
10                    Platform_Indeed 1.17
7   English Requirement in Job Ad_Yes 1.09
8     Dutch Requirement in Job Ad_Yes 1.03
6           Job Description num_words 1.01
9                   Platform_LinkedIn  NaN
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_jobs ==================================================
                Variable               VIF
9                   Platform_LinkedIn 3.70
10                    Platform_Indeed 3.60
2                         Gender_Male 2.27
1                       Gender_Female 1.95
4                         Age_Younger 1.79
5              % Sector pe

In [17]:
for df_name, df in dataframes.items():
    print('+'*120)
    print(f'{"="*50} RESULTS FOR {df_name} {"="*50}')

    if df_name == 'df_manual':
        considered_features_list = [dvs]
    elif df_name == 'df_jobs':
        considered_features_list = [dvs, dvs_prob]

    for considered_features in considered_features_list:
        vif = compute_vif(df, considered_features)
        print(vif.sort_values('VIF', ascending=False))


++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_manual ==================================================
    Variable   VIF
1      Warmth 1.00
2  Competence 1.00
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
================================================== RESULTS FOR df_jobs ==================================================
    Variable   VIF
1      Warmth 1.01
2  Competence 1.01
          Variable         VIF
2  Competence_Probability 1.02
1      Warmth_Probability 1.02
